# The Base Classification Model



In [4]:
import torch
from d2l import torch as d2l

class Classification(d2l.Module):  
    def __init__(self):
        super().__init__()

    def training_step(self, batch):
        X, y = batch
        l = self.loss(self(X), y)
        epoch = self.trainer.train_batch_idx / self.trainer.num_train_batches
        self.board.xlabel = 'epoch'
        self.board.draw(epoch, l, 'train_loss', every_n=50)
        return l

@d2l.add_to_class(Classification)  
def validation_step(self, batch):
    X, y = batch
    y_hat = self(X)
    for k, v in (('val_loss', self.loss(y_hat, y)),
                 ('val_acc', self.accuracy(y_hat, y))):
        self.board.draw(self.trainer.epoch+1, v, k,
                        every_n=self.trainer.num_val_batches)

@d2l.add_to_class(Classification)  
def configure_optimizers(self):
    return torch.optim.SGD(self.parameters(), lr=self.lr)

Compare the predicted class with the ground-truth `y` elementwise

In [5]:
@d2l.add_to_class(Classification)  
def accuracy(self, y_hat, y):
    """Compute the number of correct predictions."""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = y_hat.argmax(axis=1)
    cmp = y_hat.type(y.dtype) == y
    return cmp.type(torch.float32).mean()